In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker as mtick
plt.style.use('ggplot')
import seaborn as sns
pd.options.display.float_format = '{:,.0f}'.format
import dataframe_image as dfi
from scipy import stats
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

In [3]:
Ames = pd.read_csv('Ames_HousePrice.csv')

In [4]:
#drop extra unnamed column
Ames = Ames.drop('Unnamed: 0', axis=1)

In [5]:
Ames_Data_Types = pd.DataFrame({'Column': Ames.columns, 'Data Type': Ames.dtypes.values})
#Ames_Data_Types.to_csv('Ames_Data_Types.csv', index=False)  #Export, only needed once

In [6]:
Ames = Ames.drop_duplicates(keep = 'first')

In [7]:
#Set PID as index
Ames.set_index('PID', inplace=True, drop=False)

In [8]:
Ames_Missing_Values = (
    Ames.isnull().sum()[Ames.isnull().sum() != 0]
    .to_frame('MissingCount')
    .assign(MissingPercent=lambda x: 100 * x['MissingCount'] / len(Ames))
    .reset_index()
    .rename(columns={'index': 'ColumnName'})
)
Ames_Missing_Columns = Ames_Missing_Values['ColumnName']

In [9]:
Ames_Missing_Columns = Ames[Ames_Missing_Values['ColumnName']]
Ames_Missing_Columns_Numeric = Ames_Missing_Columns.select_dtypes(include=['number'])
Ames_Missing_Columns_Categorical = Ames_Missing_Columns.select_dtypes(include=['object', 'category'])

In [10]:
#Round 1 Imputation: Zoning & Quality Pivot Tables
Zoning_Shape_Pivot = Ames.pivot_table(
    index=['MSZoning', 'LotShape'],
    aggfunc={'LotFrontage': 'mean', 'LotArea': 'mean'}
).rename(columns={
    'LotFrontage': 'LotFrontage_Mean',
    'LotArea': 'LotArea_Mean'
})

OverallQual_Cond_Pivot = Ames.pivot_table(
    index=['OverallQual', 'OverallCond'],
    aggfunc={
        'Electrical': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan,
        'MasVnrType': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
    }
).rename(columns={
    'Electrical': 'Electrical_Mode',
    'MasVnrType': 'MasVnrType_Mode'
})

#Merge Pivots into Ames
Ames = Ames.merge(Zoning_Shape_Pivot.reset_index(), on=['MSZoning', 'LotShape'], how='left')
Ames = Ames.merge(OverallQual_Cond_Pivot.reset_index(), on=['OverallQual', 'OverallCond'], how='left')

#Impute LotFrontage proportionally to LotArea
fallback_ratio = Ames['LotFrontage'].dropna().mean() / Ames['LotArea'].dropna().mean()
Ames['LotFrontage'] = Ames['LotFrontage'].fillna(
    (Ames['LotFrontage_Mean'] / Ames['LotArea_Mean'] * Ames['LotArea']).fillna(fallback_ratio * Ames['LotArea'])
)

#Impute Electrical
Ames['Electrical'] = Ames['Electrical'].fillna(Ames['Electrical_Mode'])
Ames['Electrical'] = Ames['Electrical'].fillna(Ames['Electrical'].dropna().mode().iloc[0])

#MasVnrType: “No Veneer” where area ≤ 0, else from pivot/mode
Ames.loc[Ames['MasVnrType'].isnull() & (Ames['MasVnrArea'] <= 0), 'MasVnrType'] = 'No Veneer'
Ames['MasVnrType'] = Ames['MasVnrType'].fillna(Ames['MasVnrType_Mode'])
Ames['MasVnrType'] = Ames['MasVnrType'].fillna(Ames['MasVnrType'].dropna().mode().iloc[0])

#Clean up Round 1 merge columns
Ames.drop(['LotFrontage_Mean', 'LotArea_Mean', 'Electrical_Mode', 'MasVnrType_Mode'], axis=1, inplace=True)

#Simple Fill for Remaining NAs
Ames.fillna({
    'Alley': 'No Alley Access',
    'BsmtCond': 'No Basement',
    'BsmtExposure': 'No Basement',
    'BsmtFinType1': 'No Basement',
    'BsmtQual': 'No Basement',
    'Fence': 'No Fence',
    'FireplaceQu': 'No Fireplace',
    'GarageType': 'No Garage',
    'MiscFeature': 'No Misc Feature',
    'PoolQC': 'No Pool'
}, inplace=True)

for col in ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'MasVnrArea']:
    Ames[col] = Ames[col].fillna(0)
Ames['MasVnrArea'] = Ames['MasVnrArea'].replace({1: 0})  # correction

In [11]:
#Round 2 Imputation: Garage Pivot Tables
OverallQual_Cond_GarageType_Pivot = Ames.pivot_table(
    index=['OverallQual', 'OverallCond', 'GarageType'],
    aggfunc={
        'GarageFinish': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan,
        'GarageQual': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan,
        'GarageCond': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
    }
).rename(columns={
    'GarageFinish': 'GarageFinish_Mode',
    'GarageQual': 'GarageQual_Mode',
    'GarageCond': 'GarageCond_Mode'
})

#Merge Pivot
Ames = Ames.merge(OverallQual_Cond_GarageType_Pivot.reset_index(),
                  on=['OverallQual', 'OverallCond', 'GarageType'], how='left')

#Imputations
Ames.loc[Ames['GarageType'] == 'No Garage', ['GarageFinish', 'GarageQual', 'GarageCond']] = 'No Garage'
Ames['GarageYrBlt'] = Ames['GarageYrBlt'].fillna(Ames['YearBuilt'])
Ames['GarageFinish'] = Ames['GarageFinish'].fillna(Ames['GarageFinish_Mode'])
Ames['GarageQual'] = Ames['GarageQual'].fillna(Ames['GarageQual_Mode'])
Ames['GarageCond'] = Ames['GarageCond'].fillna(Ames['GarageCond_Mode'])

#Fallbacks
for col in ['GarageFinish', 'GarageQual', 'GarageCond']:
    Ames[col] = Ames[col].fillna(Ames[col].dropna().mode().iloc[0])

#Drop merged columns
Ames.drop(['GarageFinish_Mode', 'GarageQual_Mode', 'GarageCond_Mode'], axis=1, inplace=True)

In [12]:
#BsmtFinType2 Imputation
Ames.loc[Ames['BsmtFinType1'] == 'No Basement', 'BsmtFinType2'] = 'No Basement'
Ames['BsmtFinType2'] = Ames['BsmtFinType2'].fillna(Ames['BsmtFinType2'].dropna().mode().iloc[0])

#Round 3 Imputation: Garage & Basement Pivot Tables
Garage_Qual_Cond_Type_Pivot = Ames.pivot_table(
    index=['GarageQual', 'GarageCond', 'GarageType'],
    aggfunc={
        'GarageArea': 'mean',
        'GarageCars': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
    }
).rename(columns={
    'GarageArea': 'GarageArea_Mean',
    'GarageCars': 'GarageCars_Mode'
})

Basement_Qual_FinType1_FinType2_Pivot = Ames.pivot_table(
    index=['BsmtQual', 'BsmtFinType1', 'BsmtFinType2'],
    aggfunc={
        'BsmtFullBath': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan,
        'BsmtHalfBath': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
    }
).rename(columns={
    'BsmtFullBath': 'BsmtFullBath_Mode',
    'BsmtHalfBath': 'BsmtHalfBath_Mode'
})

#Merge both pivots
Ames = Ames.merge(Garage_Qual_Cond_Type_Pivot.reset_index(),
                  on=['GarageQual', 'GarageCond', 'GarageType'], how='left')
Ames = Ames.merge(Basement_Qual_FinType1_FinType2_Pivot.reset_index(),
                  on=['BsmtQual', 'BsmtFinType1', 'BsmtFinType2'], how='left')

#Impute numeric + categorical
Ames['GarageArea'] = Ames['GarageArea'].fillna(Ames['GarageArea_Mean'])
Ames['GarageCars'] = Ames['GarageCars'].fillna(Ames['GarageCars_Mode'])
Ames['BsmtFullBath'] = Ames['BsmtFullBath'].fillna(Ames['BsmtFullBath_Mode'])
Ames['BsmtHalfBath'] = Ames['BsmtHalfBath'].fillna(Ames['BsmtHalfBath_Mode'])

#Global fallback fill for residual NaNs
for col in ['GarageArea', 'GarageCars', 'BsmtFullBath', 'BsmtHalfBath']:
    if Ames[col].isnull().any():
        if Ames[col].dtype.kind in 'biufc':
            Ames[col] = Ames[col].fillna(Ames[col].mean())
        else:
            Ames[col] = Ames[col].fillna(Ames[col].mode().iloc[0])

#Drop temporary pivot columns
Ames.drop([
    'GarageArea_Mean', 'GarageCars_Mode',
    'BsmtFullBath_Mode', 'BsmtHalfBath_Mode'
], axis=1, inplace=True)

In [13]:
#Missing Value Check
print('\nColumns with Missing Values:\n', Ames.isnull().sum()[Ames.isnull().sum() != 0]) #sum of missing values by column name


Columns with Missing Values:
 Series([], dtype: int64)


In [14]:
Ames_Columns_Numeric = Ames[['PID', 
    '1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BsmtFinSF1', 'BsmtFinSF2',
    'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF', 'EnclosedPorch', 'Fireplaces', 'FullBath',
    'GarageArea', 'GarageCars', 'GarageYrBlt', 'GrLivArea', 'HalfBath', 'KitchenAbvGr',
    'LotArea', 'LotFrontage', 'LowQualFinSF', 'MasVnrArea', 'MiscVal', 'MoSold',
    'OpenPorchSF', 'PoolArea', 'ScreenPorch', 'TotalBsmtSF', 'TotRmsAbvGrd', 'WoodDeckSF',
    'YearBuilt', 'YearRemodAdd', 'YrSold'
]]
Ames_Columns_Ordinal = Ames[['PID', 
    'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual',
    'Electrical', 'ExterCond', 'ExterQual', 'Fence', 'FireplaceQu', 'Functional',
    'GarageCond', 'GarageFinish', 'GarageQual', 'HeatingQC', 'KitchenQual',
    'LandSlope', 'LotShape', 'OverallCond', 'OverallQual', 'PavedDrive',
    'PoolQC', 'Utilities'
]]
Ames_Columns_Nominal = Ames[['PID', 
    'Alley', 'BldgType', 'CentralAir', 'Condition1', 'Condition2',
    'Exterior1st', 'Exterior2nd', 'Foundation', 'GarageType', 'Heating',
    'HouseStyle', 'LandContour', 'LotConfig', 'MasVnrType', 'MiscFeature',
    'MSSubClass', 'MSZoning', 'Neighborhood', 'RoofMatl', 'RoofStyle',
    'SaleCondition', 'SaleType', 'Street'
]].astype(str)

In [15]:
value_count_list = []
for col in Ames_Columns_Ordinal.columns:
    counts = Ames_Columns_Ordinal[col].value_counts(dropna=False)
    temp_df = pd.DataFrame({
        'Column Name': col,
        'Value': counts.index,
        'Value Count': counts.values
    })
    value_count_list.append(temp_df)
Ames_Ordinal_ValueCounts = pd.concat(value_count_list, ignore_index=True)
#Ames_Ordinal_ValueCounts.to_csv('Ames_Ordinal_ValueCounts.csv', index=False)

In [16]:
#Import Ordinal Value Ordering Legend created based off of Ames_Ordinal_ValueCounts export
Ames_Ordinal_Legend = pd.read_csv('Ames_Ordinal_Legend.csv')

#Convert already numeric ordinal values to string in original Dataset
ordinal_cols = Ames_Ordinal_Legend['Name'].unique()
for col in ordinal_cols:
    if col in Ames.columns:
        Ames[col] = Ames[col].astype(str)
# Loop through columns that are in the ordinal legend
for col in Ames.columns:
    if col in Ames_Ordinal_Legend['Name'].unique():
        # Create the mapping dictionary for this column
        mapping_dict = Ames_Ordinal_Legend.loc[Ames_Ordinal_Legend['Name'] == col,
                                               ['Value', 'Value_Order']].set_index('Value')['Value_Order'].to_dict()
        # Map values in Ames[col] using this dictionary
        Ames[col] = Ames[col].map(mapping_dict)

#Missing Value Check
print('\nColumns with Missing Values:\n', Ames.isnull().sum()[Ames.isnull().sum() != 0]) #sum of missing values by column name
print('\nRows with Missing Values:\n', Ames[Ames.isnull().any(axis=1)]) #show rows with missing values

# Check the data types and unique values
ordinal_cols = Ames_Ordinal_Legend['Name'].unique() # List of ordinal columns
# Check dtype and unique values as plain Python ints
for col in ordinal_cols:
    if col in Ames.columns:
        unique_vals = [int(v) for v in sorted(Ames[col].dropna().unique())]
        print(f"{col}: dtype={Ames[col].dtype}, unique values={unique_vals}")


Columns with Missing Values:
 Series([], dtype: int64)

Rows with Missing Values:
 Empty DataFrame
Columns: [PID, GrLivArea, SalePrice, MSSubClass, MSZoning, LotFrontage, LotArea, Street, Alley, LotShape, LandContour, Utilities, LotConfig, LandSlope, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, OverallQual, OverallCond, YearBuilt, YearRemodAdd, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, MasVnrArea, ExterQual, ExterCond, Foundation, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinSF1, BsmtFinType2, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, Heating, HeatingQC, CentralAir, Electrical, 1stFlrSF, 2ndFlrSF, LowQualFinSF, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, KitchenQual, TotRmsAbvGrd, Functional, Fireplaces, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageCars, GarageArea, GarageQual, GarageCond, PavedDrive, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, PoolQC, Fence, MiscFeature, Mis

In [17]:
#Update with new Ames Data
Ames_Columns_Ordinal = Ames[['PID', 
    'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual',
    'Electrical', 'ExterCond', 'ExterQual', 'Fence', 'FireplaceQu', 'Functional',
    'GarageCond', 'GarageFinish', 'GarageQual', 'HeatingQC', 'KitchenQual',
    'LandSlope', 'LotShape', 'OverallCond', 'OverallQual', 'PavedDrive',
    'PoolQC', 'Utilities'
]]
#Combination Table of Numeric & Ordinal Columns
Ames_Columns_Numeric_Ordinal = Ames[
    list(Ames_Columns_Numeric.columns) + list(Ames_Columns_Ordinal.columns)
]
#Combination Table of Nominal & Ordinal Columns
Ames_Columns_Nominal_Ordinal = Ames[
    list(Ames_Columns_Nominal.columns) + list(Ames_Columns_Ordinal.drop('PID', axis=1).columns)
]

In [18]:
Y = Ames['SalePrice']
lm = LinearRegression()
kf = KFold(n_splits=5, shuffle=True, random_state=2)
encoder = OneHotEncoder(drop='first', sparse_output=False)

In [20]:
def MLR_CV(Nominal_cols, Ordinal_cols, Numeric_cols):
    # --- One-Hot Encode Nominal ---
    Ames_Optimal_Nominal_One_Hot = encoder.fit_transform(Ames[Nominal_cols])
    Ames_Optimal_Nominal_One_Hot_df = pd.DataFrame(
        Ames_Optimal_Nominal_One_Hot,
        columns=encoder.get_feature_names_out(Nominal_cols),
        index=Ames.index
    )

    # --- One-Hot Encode Ordinal ---
    Ames_Optimal_Ordinal_One_Hot = encoder.fit_transform(Ames[Ordinal_cols])
    Ames_Optimal_Ordinal_One_Hot_df = pd.DataFrame(
        Ames_Optimal_Ordinal_One_Hot,
        columns=encoder.get_feature_names_out(Ordinal_cols),
        index=Ames.index
    )

    # --- Merge Nominal & Ordinal Encodings ---
    Ames_Optimal_Categorical_One_Hot = pd.merge(
        Ames_Optimal_Nominal_One_Hot_df,
        Ames_Optimal_Ordinal_One_Hot_df,
        left_index=True,
        right_index=True,
        how='left'
    )

    # --- Merge with Numeric Features ---
    Ames_MLR_Optimal_Columns = pd.merge(
        Ames[Numeric_cols],
        Ames_Optimal_Categorical_One_Hot,
        left_index=True,
        right_index=True,
        how='left'
    ).drop('PID', axis=1)

    # --- Model & Evaluation ---
    lm = LinearRegression()
    kf = KFold(n_splits=5, shuffle=True, random_state=2)

    X = Ames_MLR_Optimal_Columns
    lm.fit(X, Y)
    r2 = cross_val_score(lm, X, Y, cv=kf, scoring='r2')

    # --- Output ---
    print(f'Nominal: {Nominal_cols}')
    print(f'Ordinal: {Ordinal_cols}')
    print(f'Numeric: {Numeric_cols}\n')
    print(f"R² per fold (sorted): {', '.join(f'{score:.4f}' for score in sorted(r2))}")
    print(f'R² Average: {r2.mean():.4f}')
    print(f'Intercept: {lm.intercept_:,.2f}')
#    print('Coefficients:')
#    for name, coef in zip(X.columns, lm.coef_):
#        print(f'  {name:<22} {coef:,.0f}')

In [21]:
print('---MLR MODEL WITH ALL FEATURES---')
MLR_CV(Ames_Columns_Nominal.columns, Ames_Columns_Ordinal.columns, Ames_Columns_Numeric.columns)

---MLR MODEL WITH ALL FEATURES---
Nominal: Index(['PID', 'Alley', 'BldgType', 'CentralAir', 'Condition1', 'Condition2',
       'Exterior1st', 'Exterior2nd', 'Foundation', 'GarageType', 'Heating',
       'HouseStyle', 'LandContour', 'LotConfig', 'MasVnrType', 'MiscFeature',
       'MSSubClass', 'MSZoning', 'Neighborhood', 'RoofMatl', 'RoofStyle',
       'SaleCondition', 'SaleType', 'Street'],
      dtype='object')
Ordinal: Index(['PID', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'BsmtQual', 'Electrical', 'ExterCond', 'ExterQual', 'Fence',
       'FireplaceQu', 'Functional', 'GarageCond', 'GarageFinish', 'GarageQual',
       'HeatingQC', 'KitchenQual', 'LandSlope', 'LotShape', 'OverallCond',
       'OverallQual', 'PavedDrive', 'PoolQC', 'Utilities'],
      dtype='object')
Numeric: Index(['PID', '1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF',
       'EnclosedPorch', 'Fireplaces', '